In [8]:
# IMPORT LIBRIES
import time
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn.preprocessing import OrdinalEncoder

# Basic classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# Ensemble classifiers
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, VotingClassifier, StackingClassifier

# Boosting libraries
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier



In [9]:
# IMPORT DATASET
df = pd.read_csv('https://raw.githubusercontent.com/rohitmande-inttrvu/finance_loan_approval/refs/heads/main/Finance.csv')
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [10]:
# DROP LOAD_ID
df.drop(columns=['Loan_ID'],inplace=True)

In [11]:
df['Loan_Status'].replace({'Y':1,'N':0},inplace=True)

C:\Users\Yogesh\AppData\Local\Temp\ipykernel_26764\125627036.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Loan_Status'].replace({'Y':1,'N':0},inplace=True)
C:\Users\Yogesh\AppData\Local\Temp\ipykernel_26764\125627036.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Loan_Status'].re

In [12]:
classifiers = {
    'Logistic Regression': LogisticRegression(),
    'KNN': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC(),
    'Gaussian': GaussianNB(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost': XGBClassifier(),
    'LightGBM': LGBMClassifier()
}

In [13]:
#CONFIGURE EXPERIMENT
import mlflow
import dagshub
dagshub.init(repo_owner='yogibaba7', repo_name='loan_approval_prediction', mlflow=True)
# set tracking uri
mlflow.set_tracking_uri('https://dagshub.com/yogibaba7/loan_approval_prediction.mlflow/')
# create a experiment
mlflow.set_experiment('exp4_bestmodel')

Initialized MLflow to track repo "yogibaba7/loan_approval_prediction"

Repository yogibaba7/loan_approval_prediction initialized!

<Experiment: artifact_location='mlflow-artifacts:/95242b81f76f43a8a8964f277cdd7dde', creation_time=1744463603346, experiment_id='3', last_update_time=1744463603346, lifecycle_stage='active', name='exp4_bestmodel', tags={}>

In [14]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

with mlflow.start_run(nested=True,description='this experiment is created for find the  best scaler'):
    X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Loan_Status']),df['Loan_Status'],test_size=0.2,random_state=42)
    # CATEGORICAL COLUMNS AND NUMERICAL COLUMNS
    cat_cols = []
    num_cols = []
    columns = X_train.columns
    for col in columns:
        if df[col].nunique()<5:
            cat_cols.append(col)
        else:
            num_cols.append(col)
    print(f"Categorical cols : {cat_cols}")
    print(f"Numerical cols : {num_cols}")

    # encoding
    oe = OrdinalEncoder()
    X_train[cat_cols] = oe.fit_transform(X_train[cat_cols])
    X_test[cat_cols] = oe.transform(X_test[cat_cols])

    # iterative imputer
    iterativeimputer = IterativeImputer()
    X_train = iterativeimputer.fit_transform(X_train)
    X_test = iterativeimputer.transform(X_test)

    X_train = pd.DataFrame(X_train,columns=columns)
    X_test = pd.DataFrame(X_test,columns=columns)

    # scaling
    scaler = RobustScaler()
    X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
    X_test[num_cols] = scaler.transform(X_test[num_cols])

    
    for classifier_name,classifier in classifiers.items():
        with mlflow.start_run(nested=True,description=f"in this run {classifier} trained"):

            # log imputer 
            mlflow.log_param('imputer','iterativeimputer')
            mlflow.log_param('scaler',scaler)
            mlflow.log_param(classifier_name,classifier)

            # model training
            classifier.fit(X_train,y_train)
            y_pred = classifier.predict(X_test)

            
            # log model
            mlflow.sklearn.log_model(classifier,classifier_name)

            # model evaluations
            accuracy = accuracy_score(y_test,y_pred)
            precision = precision_score(y_test,y_pred)
            recall = recall_score(y_test,y_pred)
            f1 = f1_score(y_test,y_pred)

            # log metrics 
            mlflow.log_metric('accuracy',accuracy)
            mlflow.log_metric('precision',precision)
            mlflow.log_metric('recall',recall)
            mlflow.log_metric('f1',f1)

            # Save and log the notebook
            import os
            notebook_path = "exp1_baseline_model.ipynb"
            os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
            mlflow.log_artifact(notebook_path)

            
            print(f"accuracy score : {accuracy}")
            print(f"precision score : {precision}")
            print(f"recall score : {recall}")
            print(f"f1 score : {f1}")     
            print("---------------------------------------------------------")

Categorical cols : ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Credit_History', 'Property_Area']
Numerical cols : ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']


c:\Users\Yogesh\Desktop\loan_approval_prediction\myvenv\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
2025/04/13 10:18:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


accuracy score : 0.7886178861788617
precision score : 0.7596153846153846
recall score : 0.9875
f1 score : 0.8586956521739131
---------------------------------------------------------
🏃 View run popular-shoat-546 at: https://dagshub.com/yogibaba7/loan_approval_prediction.mlflow/#/experiments/3/runs/beef3c74e3644aea8a44ef5a4d3b0000
🧪 View experiment at: https://dagshub.com/yogibaba7/loan_approval_prediction.mlflow/#/experiments/3


2025/04/13 10:18:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


accuracy score : 0.6666666666666666
precision score : 0.6893203883495146
recall score : 0.8875
f1 score : 0.7759562841530054
---------------------------------------------------------
🏃 View run beautiful-robin-510 at: https://dagshub.com/yogibaba7/loan_approval_prediction.mlflow/#/experiments/3/runs/44a0d24bf2bd421b9cdc89a3e625797f
🧪 View experiment at: https://dagshub.com/yogibaba7/loan_approval_prediction.mlflow/#/experiments/3


...

accuracy score : 0.7804878048780488
precision score : 0.7623762376237624
recall score : 0.9625
f1 score : 0.850828729281768
---------------------------------------------------------
🏃 View run delightful-asp-902 at: https://dagshub.com/yogibaba7/loan_approval_prediction.mlflow/#/experiments/3/runs/2f55bbd225d84200a0a54b719f706ca7
🧪 View experiment at: https://dagshub.com/yogibaba7/loan_approval_prediction.mlflow/#/experiments/3


2025/04/13 10:22:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


accuracy score : 0.7723577235772358
precision score : 0.76
recall score : 0.95
f1 score : 0.8444444444444444
---------------------------------------------------------
🏃 View run abundant-cod-854 at: https://dagshub.com/yogibaba7/loan_approval_prediction.mlflow/#/experiments/3/runs/34a4e6523e314afd9ddcd16bd6518458
🧪 View experiment at: https://dagshub.com/yogibaba7/loan_approval_prediction.mlflow/#/experiments/3
[LightGBM] [Info] Number of positive: 342, number of negative: 149
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001044 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 406
[LightGBM] [Info] Number of data points in the train set: 491, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.696538 -> initscore=0.830864
[LightGBM] [Info] Start training from score 0.830864
[LightGBM] [Warning] No further spli

2025/04/13 10:22:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


accuracy score : 0.7804878048780488
precision score : 0.7676767676767676
recall score : 0.95
f1 score : 0.8491620111731844
---------------------------------------------------------
🏃 View run gaudy-koi-621 at: https://dagshub.com/yogibaba7/loan_approval_prediction.mlflow/#/experiments/3/runs/1ce83d20fa4140b4a9fa25fb930f1411
🧪 View experiment at: https://dagshub.com/yogibaba7/loan_approval_prediction.mlflow/#/experiments/3
🏃 View run sedate-perch-475 at: https://dagshub.com/yogibaba7/loan_approval_prediction.mlflow/#/experiments/3/runs/ccd423c4428f4c178464ab1b62ddb217
🧪 View experiment at: https://dagshub.com/yogibaba7/loan_approval_prediction.mlflow/#/experiments/3
